Выделение и использование лингвистических признаков
==============
20.10.2022

Выделение и генерация текста с помощью тегов частей речи

In [110]:
import spacy as sp
nlp = sp.load('ru_core_news_lg')

#### Теги для чисел, символов и знаков препинания

In [111]:
doc = nlp(u"Привет, вы должны мне $12 тысяч. ")
for token in doc:
    print(token.text, token.pos_,sp.explain(token.pos_))

# explain(token.pos_) возвращает описание токена

Привет NOUN noun
, PUNCT punctuation
вы PRON pronoun
должны ADJ adjective
мне PRON pronoun
$ SYM symbol
12 NUM numeral
тысяч NOUN noun
. PUNCT punctuation


Второй и третий столбцы содержат теги общих и уточненных частей речи соответственно. В четвертом столбце приведено описание тегов уточненных частей речи из третьего столбца.

In [112]:
doc = nlp(u"Вы должны мне $12 миллионов. ")
for token in doc:
    print(token.text, token.pos_, token.tag_)

Вы PRON PRON
должны ADJ ADJ
мне PRON PRON
$ SYM SYM
12 NUM NUM
миллионов NOUN NOUN
. PUNCT PUNCT


Почему pos_ и tag_ совпадают в русском языке????????????????????????????????/s


In [113]:
nlp_en = sp.load('en_core_web_sm')
doc_en = nlp_en(u'The firm earned $1.5 million in 2017.')

for token in doc_en:
    print(token.text, token.pos_, token.tag_)


The DET DT
firm NOUN NN
earned VERB VBD
$ SYM $
1.5 NUM CD
million NUM CD
in ADP IN
2017 NUM CD
. PUNCT .


##### Преобразование утвердительных высказываний
##### в вопросительные



Допустим, наше NLP-приложение должно генерировать вопросы на
основе полученных утвердительных высказываний.

In [114]:
doc = nlp(u"Я умею водить машину.")
for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_, token.head.text, sp.explain(token.dep_))


Я PRON PRON nsubj умею nominal subject
умею VERB VERB ROOT умею root
водить VERB VERB xcomp умею open clausal complement
машину NOUN NOUN obj водить object
. PUNCT PUNCT punct умею punctuation


In [115]:
sent=''
for i, token in enumerate(doc):
    if token.pos_ == "PRON":
        sent+='Вы '
    elif token.text == ".":
        sent+='?'
    else:
        sent+=token.text + ' '


sent

'Вы умею водить машину ?'

In [140]:
doc = nlp(u"Я хочу красную грушу.")
for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_, token.head.text, sp.explain(token.dep_))


Я PRON PRON nsubj хочу nominal subject
хочу VERB VERB ROOT хочу root
красную ADJ ADJ amod грушу adjectival modifier
грушу NOUN NOUN obj хочу object
. PUNCT PUNCT punct хочу punctuation


In [141]:
def find_chunk(doc):
    chunk = ''
    for i,token in enumerate(doc):
        if token.dep_ == 'obj':
            shift = len([w for w in  token.children])
            chunk = doc[i-shift:i+2]
            break
    return chunk

In [142]:
s=find_chunk(doc)
s

красную грушу.

In [143]:
def determine_question_type(chunk):
    question_type = 'yesno'
    for token in chunk:
        if token.dep_ == 'amod':
            question_type = 'info'
    return question_type

In [144]:
type = determine_question_type(s)

In [166]:
def generate_question(doc, question_type):
    sent = ''
    for i, token in enumerate(doc):
        if token.pos_ == "PRON":
            sent+='Вы '
        elif token.text == ".":
            sent+='?'
        else:
            sent+=token.text + ' '

    doc=nlp(sent)
    if question_type == 'info':
        for i,token in enumerate(doc):
            if token.dep_ == 'obj':
                shift = len([w for w in  token.children])
                chunk = doc[i-shift:i+2]
                sent = 'Зачем вам ' + chunk.text 
                break
    if question_type == 'yesno':
        for i,token in enumerate(doc):
            if token.dep_ == 'dobj':
                sent = doc[:i-1].text + ' a red ' + doc[i:].text
                break
    doc=nlp(sent)


    return sent

In [176]:


doc = nlp(u"Я хочу домой.")
type = determine_question_type(s)
print(type)
generate_question(doc, type)

info


'Вы хочу домой ?'